In [13]:
from twython import Twython, TwythonRateLimitError, TwythonError
from textblob import TextBlob
from glob import glob
from csv import DictReader, DictWriter
import pandas as pd
import os, re, string

In [5]:
ConsumerKey = 'Xap2Jytwc3ehirzhWhPtIg' 
ConsumerSecret = 'M19t2uyoXSDnNxH01c4EAWvoAScpSyUlBHiGQx4U'
AccessToken = '166071132-r247QGhTdb9a1SdtH36avAndjAjwJksdJZLLqDYt'
AccessSecret = 'y7CGzzZzwyO72sx7yNedrS3hNXTAPWk5uqjDVdczykaKc'

twitter_user = 'HassanNisar'
tw = Twython(ConsumerKey, ConsumerSecret, AccessToken, AccessSecret)

In [6]:
def timeline(user_id, max_id=None):    
    for i in range(16):
        tweets = tw.get_user_timeline(screen_name=user_id,
                                      max_id=max_id,
                                      count=200,
                                      trim_user=True,
                                      exclude_replies=True,
                                      include_rts=False,
                                      tweet_mode='extended')
        if len(tweets) > 0:  # last page should have zero results
            for tweet in tweets:
                max_id = tweet['id'] - 1
                yield tweet
        else:
            break

In [14]:
# https://gist.github.com/emallson/9e56a99973b3091124cd
# I have a directory of csv files containing user profiles. 
# Pull the IDs from these.
fields = ['id', 'user', 'created_at', 'lang', 'full_text']
#fields = ['contributors', 'coordinates', 'created_at', 'display_text_range',
#        'entities', 'extended_entities', 'favorite_count', 'favorited',
#        'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
#        'in_reply_to_status_id', 'in_reply_to_status_id_str',
#        'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
#        'lang', 'place', 'possibly_sensitive', 'quoted_status',
#        'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
#        'retweet_count', 'retweeted', 'source', 'truncated', 'user']

users_list = open("user.txt", "r")
for name in users_list:
    path = "timelines/{}.csv".format(name.rstrip())
    if os.path.exists(path):
        continue # skip users that have already been read.
    with open(path, "w", encoding='utf-8', newline='') as timecsv:
        print(name)
        writer = DictWriter(timecsv, fields)
        writer.writeheader()
        # max_id is used to continue a user's timeline from 
        # the last tweet read in the event that we get 
        # rate-limited in the middle of a user's timeline
        max_id = None
        while True:
            try:
                for tweet in timeline(name, max_id):
                    sub = {k: v for k, v in tweet.items()
                           if k in fields}
                    sub['user'] = sub['user']['id']
                    sub['full_text'] = sub['full_text'].rstrip()
                    max_id = sub['id']
                    if (sub['lang'] == 'ur'):
                        writer.writerow(sub)
                max_id = None
                break
            except TwythonRateLimitError as e:
                sleep_until(e.retry_after) # sleep until the given date
            except TwythonError as e:
                # I *think* this is caused by protected profiles. 
                # I can read some user profile info, but not the timeline
                print(e)
                print("Skipping...")

In [189]:
# document = pd.read_csv('timelines/HassanNisar.csv')

df = pd.concat([pd.read_csv(f, encoding='utf-8') for f in glob('timelines/*.csv')], ignore_index=True)

In [190]:
df = df.replace(r'\n', '', regex = True)

In [191]:
df['text'] = df['full_text'].apply(lambda x: re.sub(r'https?://[A-Za-z0-9./]+','',x))

In [192]:
df['text'] = df['text'].apply(lambda x: re.sub(r'[a-z]','',x,flags=re.I))

In [193]:
df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('','',string.digits)))

In [194]:
df['text'] = df['text'].apply(lambda x: re.sub(r'[،,?!٪\'“”؟ھ]+\ *',' ',x))

In [197]:
import emoji
def give_emoji_free_text(text):
    allchars = [strs for strs in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

df['text'] = df['text'].apply(lambda x: give_emoji_free_text(x))

In [150]:
# df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

In [202]:
texts = df['text']
labels = df['user']

In [203]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

In [204]:
# split the dataset into training and validation datasets 
train_x, test_x, train_y, test_y = model_selection.train_test_split(texts, labels, train_size=0.75)

C:\Users\Nabeel\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [205]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [206]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [209]:
cnt_vectorizer = CountVectorizer()
features_c = cnt_vectorizer.fit_transform(texts)
features_c_nd = features_c.toarray()
features_c_nd.shape

(4962, 13425)

In [216]:
print(features_c)

  (0, 100)	1
  (0, 9872)	1
  (0, 6415)	1
  (0, 5711)	1
  (0, 5341)	1
  (0, 4138)	1
  (0, 7974)	1
  (0, 12217)	1
  (0, 12233)	1
  (0, 5227)	1
  (0, 7769)	1
  (0, 1075)	1
  (0, 11542)	1
  (0, 5236)	1
  (0, 10891)	1
  (0, 2944)	1
  (0, 2953)	1
  (0, 5621)	1
  (0, 9421)	1
  (1, 4140)	1
  (1, 2242)	1
  (1, 12990)	1
  (1, 11797)	1
  (1, 13147)	1
  (1, 13398)	1
  :	:
  (4959, 6462)	1
  (4959, 7015)	1
  (4959, 4217)	1
  (4959, 9056)	1
  (4959, 3333)	2
  (4959, 3841)	1
  (4959, 8537)	1
  (4959, 9043)	1
  (4959, 11958)	2
  (4960, 4519)	1
  (4960, 13326)	1
  (4960, 582)	1
  (4960, 3308)	1
  (4960, 10153)	1
  (4960, 6271)	1
  (4960, 11958)	1
  (4960, 5621)	1
  (4961, 7910)	1
  (4961, 9303)	1
  (4961, 4412)	1
  (4961, 7678)	1
  (4961, 2183)	1
  (4961, 9416)	1
  (4961, 11651)	1
  (4961, 13059)	1


In [219]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word', max_features=5000)
features = tfidf_vectorizer.fit_transform(texts)
features_nd = features.toarray()
features_nd.shape

(4962, 5000)

In [220]:
tfidf_vectorizer.get_feature_names()

['آؤ',
 'آؤٹ',
 'آئ',
 'آئل',
 'آئندہ',
 'آئی',
 'آئین',
 'آئینی',
 'آئیں',
 'آئیے',
 'آئے',
 'آئےہیں',
 'آبائی',
 'آباد',
 'آبادی',
 'آبدیدہ',
 'آبرو',
 'آبی',
 'آتا',
 'آتی',
 'آتے',
 'آج',
 'آجائیں',
 'آجائے',
 'آجکل',
 'آخر',
 'آخری',
 'آد',
 'آدم',
 'آدمی',
 'آر',
 'آرائیاں',
 'آرائیوں',
 'آراو',
 'آرزو',
 'آرمی',
 'آرٹیکل',
 'آرڈر',
 'آرہا',
 'آرہی',
 'آرہے',
 'آزاد',
 'آزادی',
 'آزما',
 'آزمائش',
 'آسان',
 'آسانی',
 'آسمان',
 'آسماں',
 'آسٹریلوی',
 'آسٹریلیا',
 'آسیہ',
 'آشنا',
 'آشکار',
 'آصف',
 'آغاز',
 'آف',
 'آفت',
 'آفتاب',
 'آفر',
 'آفریدی',
 'آفس',
 'آفیسر',
 'آفیشل',
 'آقاؤں',
 'آل',
 'آلودگی',
 'آمادہ',
 'آمد',
 'آمدن',
 'آمدید',
 'آمریت',
 'آمنے',
 'آموز',
 'آمین',
 'آن',
 'آنا',
 'آنجہانی',
 'آنسو',
 'آنک',
 'آنیوالی',
 'آنے',
 'آنےوالی',
 'آو',
 'آواز',
 'آور',
 'آٹ',
 'آپ',
 'آپریشن',
 'آپس',
 'آپوزیشن',
 'آپکا',
 'آپکو',
 'آپکی',
 'آپکے',
 'آڈٹ',
 'آڑ',
 'آکر',
 'آگ',
 'آگئی',
 'آگئیں',
 'آگئے',
 'آگاہ',
 'آگیا',
 'آگے',
 'آہ',
 'آہی',
 'آیا',
 'ئی',
 'ئے',
 'اؤ',


In [199]:
df['text'][5]

'تمام اہلیانِ پی ایس ایل کو چاند رات مبارک ہو کل انشاءاللہ سٹیج ب ی سجے گا سیٹی ب ی بجے گی اور دیوانوں کا ک یل ب ی ہوگا # #'

In [201]:
df['full_text'][5]

'تمام اہلیانِ پی ایس ایل کو چاند رات مبارک ہو!کل انشاءاللہ سٹیج بھی سجے گا سیٹی بھی بجے گی اور دیوانوں کا کھیل بھی ہوگا!! #KhelDeewanoKa #PSL4'

In [200]:
df['text'][5].split()

['تمام',
 'اہلیانِ',
 'پی',
 'ایس',
 'ایل',
 'کو',
 'چاند',
 'رات',
 'مبارک',
 'ہو',
 'کل',
 'انشاءاللہ',
 'سٹیج',
 'ب',
 'ی',
 'سجے',
 'گا',
 'سیٹی',
 'ب',
 'ی',
 'بجے',
 'گی',
 'اور',
 'دیوانوں',
 'کا',
 'ک',
 'یل',
 'ب',
 'ی',
 'ہوگا',
 '#',
 '#']